### 0. 패키지 및 라이브러리, API KEY

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
! pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.1 MB/s eta 0:00:00


In [ ]:
! pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_upstage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm  # 진행률 표시를 위한 라이브러리
import time

In [ ]:
from openai import OpenAI# openai==1.2.0
import os
import openai
from langchain.embeddings import OpenAIEmbeddings

from langchain.vectorstores import Pinecone

from pinecone import Pinecone, ServerlessSpec # 이후 버전

from langchain.schema import Document
from langchain.schema import SystemMessage, HumanMessage
from langchain.llms.base import LLM
from typing import Optional, List
from langchain.schema import (
    Generation,
    LLMResult
)
from langchain.embeddings.base import Embeddings
from langchain_upstage import ChatUpstage
from langchain_upstage import UpstageEmbeddings

In [ ]:
# API KEY 설정
MY_PINECONE_API_KEY = "MY_PINECONE_API_KEY"
MY_UPSTAGE_API_KEY = "MY_UPSTAGE_API_KEY"

### 1. 데이터 로드

In [ ]:
data = pd.read_csv('/content/imdb-movies-dataset.csv', sep=',')

# 데이터 확인
data.head()

,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Poster          10000 non-null  object 
 1   Title           10000 non-null  object 
 2   Year            9850 non-null   float64
 3   Certificate     7370 non-null   object 
 4   Duration (min)  9664 non-null   float64
 5   Genre           9993 non-null   object 
 6   Rating          9596 non-null   float64
 7   Metascore       7555 non-null   float64
 8   Director        9995 non-null   object 
 9   Cast            9961 non-null   object 
 10  Votes           9596 non-null   object 
 11  Description     10000 non-null  object 
 12  Review Count    9999 non-null   object 
 13  Review Title    9483 non-null   object 
 14  Review          9484 non-null   object 
dtypes: float64(4), object(11)
memory usage: 1.1+ MB


In [ ]:
data.shape

(10000, 15)

In [ ]:
# 필요한 컬럼 선택
df = data[['Title', 'Year', 'Genre', 'Description', 'Director', 'Cast', 'Review']]

# 결측치 제거
df.dropna(inplace=True)

# 데이터 샘플 확인
df.head()

<ipython-input-10-f7b6e412e8d5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,Title,Year,Genre,Description,Director,Cast,Review
0,The Idea of You,2023.0,"Comedy, Drama, Romance","Solène, a 40-year-old single mom, begins an un...",Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","This film, as well as the reaction to it, is a..."
1,Kingdom of the Planet of the Apes,2023.0,"Action, Adventure, Sci-Fi","Many years after the reign of Caesar, a young ...",Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","I'm a big fan of all the planet of the apes, a..."
2,Unfrosted,2023.0,"Biography, Comedy, History","In 1963 Michigan, business rivals Kellogg's an...",Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",Pretty much the worst criticism you can lay on...
3,The Fall Guy,2023.0,"Action, Comedy, Drama",A down-and-out stuntman must find the missing ...,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",Just got out of the Austin premier at SXSW and...
4,Challengers,2023.0,"Drama, Romance, Sport","Tashi, a former tennis prodigy turned coach, t...",Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",This is a tough one. I liked the concept and t...


### 2. Upstage API 키 설정

In [ ]:
# Upstage API 키 설정
# MY_UPSTAGE_API_KEY = os.environ.get(MY_UPSTAGE_API_KEY)  # 환경 변수에서 가져오기

# Upstage 클라이언트 초기화
client = OpenAI(
    api_key= MY_UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

### 3. 영화 설명에 대한 임베딩 생성

- 임베딩하는데 $0.45 사용됨

In [ ]:
# 임베딩을 저장할 리스트
embeddings = []

# 각 행에 대해 텍스트 결합 및 임베딩 생성
for idx, row in tqdm(df.iterrows(), total=df.shape[0]): # 'index'를 'idx'로 변경
    # 텍스트 결합
    text = f"Title: {row['Title']}\nYear: {row['Year']}\nGenre: {row['Genre']}\nDescription: {row['Description']}\nDirector: {row['Director']}\nCast: {row['Cast']}\nReview: {row['Review']}"

    # 임베딩 생성
    embedding = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=text
    ).data[0].embedding

    embeddings.append(embedding)

    # 0.2초 대기(API호출량 과부하 방지)
    time.sleep(0.2)

# 임베딩을 데이터프레임에 추가
df['Embeddings'] = embeddings

 81%|████████  | 7577/9342 [1:48:12<24:38,  1.19it/s]

In [ ]:
# 첫 번째 임베딩 벡터의 길이를 통해 차원 수 확인
embedding_dimension = len(embeddings[0])
print(f"Embedding Dimension: {embedding_dimension}")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# df.to_csv('/content/imdb-movies-dataset2.csv')

### 3.5 데이터 재로드(여기부터 시작)

In [ ]:
df = pd.read_csv('/content/imdb-movies-dataset2.csv', sep=',')

# 데이터 확인
df.head()

,Unnamed: 0,Title,Year,Genre,Description,Director,Cast,Review,Embeddings
0,0,The Idea of You,2023.0,"Comedy, Drama, Romance","Solène, a 40-year-old single mom, begins an un...",Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","This film, as well as the reaction to it, is a...","[0.01253509521484375, 0.006870269775390625, 0...."
1,1,Kingdom of the Planet of the Apes,2023.0,"Action, Adventure, Sci-Fi","Many years after the reign of Caesar, a young ...",Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","I'm a big fan of all the planet of the apes, a...","[-0.011474609375, -0.0159454345703125, -0.0129..."
2,2,Unfrosted,2023.0,"Biography, Comedy, History","In 1963 Michigan, business rivals Kellogg's an...",Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",Pretty much the worst criticism you can lay on...,"[-0.0018987655639648438, 0.005340576171875, 0...."
3,3,The Fall Guy,2023.0,"Action, Comedy, Drama",A down-and-out stuntman must find the missing ...,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",Just got out of the Austin premier at SXSW and...,"[-0.0156707763671875, -0.02117919921875, -0.00..."
4,4,Challengers,2023.0,"Drama, Romance, Sport","Tashi, a former tennis prodigy turned coach, t...",Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",This is a tough one. I liked the concept and t...,"[-0.022705078125, -0.0107574462890625, -0.0059..."


In [ ]:
df.shape

(9342, 9)

In [ ]:
embeddings = df['Embeddings']

# 첫 번째 임베딩 벡터의 길이를 통해 차원 수 확인
embedding_dimension = len(embeddings[0])
print(f"Embedding Dimension: {embedding_dimension}")

Embedding Dimension: 84820


In [ ]:
df.info() # Embeddings를 다시 불러온 결과 object 값으로 인식하여 차원이 84820으로 인식 됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9342 entries, 0 to 9341
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   9342 non-null   int64  
 1   Title        9342 non-null   object 
 2   Year         9342 non-null   float64
 3   Genre        9342 non-null   object 
 4   Description  9342 non-null   object 
 5   Director     9342 non-null   object 
 6   Cast         9342 non-null   object 
 7   Review       9342 non-null   object 
 8   Embeddings   9342 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 657.0+ KB


- 7분 걸림

In [ ]:
# 임베딩 벡터 문자열을 리스트로 변환
df['Embeddings'] = df['Embeddings'].apply(ast.literal_eval)

# 임베딩 벡터 리스트를 가져옵니다.
embeddings = df['Embeddings'].tolist()

# 첫 번째 임베딩 벡터의 차원 확인
embedding_dimension = len(embeddings[0])
print(f"Embedding Dimension: {embedding_dimension}")

Embedding Dimension: 4096


In [ ]:
df.head()

,Unnamed: 0,Title,Year,Genre,Description,Director,Cast,Review,Embeddings
0,0,The Idea of You,2023.0,"Comedy, Drama, Romance","Solène, a 40-year-old single mom, begins an un...",Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","This film, as well as the reaction to it, is a...","[0.01253509521484375, 0.006870269775390625, 0...."
1,1,Kingdom of the Planet of the Apes,2023.0,"Action, Adventure, Sci-Fi","Many years after the reign of Caesar, a young ...",Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","I'm a big fan of all the planet of the apes, a...","[-0.011474609375, -0.0159454345703125, -0.0129..."
2,2,Unfrosted,2023.0,"Biography, Comedy, History","In 1963 Michigan, business rivals Kellogg's an...",Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",Pretty much the worst criticism you can lay on...,"[-0.0018987655639648438, 0.005340576171875, 0...."
3,3,The Fall Guy,2023.0,"Action, Comedy, Drama",A down-and-out stuntman must find the missing ...,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",Just got out of the Austin premier at SXSW and...,"[-0.0156707763671875, -0.02117919921875, -0.00..."
4,4,Challengers,2023.0,"Drama, Romance, Sport","Tashi, a former tennis prodigy turned coach, t...",Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",This is a tough one. I liked the concept and t...,"[-0.022705078125, -0.0107574462890625, -0.0059..."


### 4. Pinecone 초기화 및 인덱스 설정

In [ ]:
# 임베딩 벡터의 차원 확인
embedding_dimension = len(embeddings[0])
print(f"Embedding Dimension: {embedding_dimension}")

# Pinecone API 키 설정

# MY_PINECONE_API_KEY = os.environ.get(MY_PINECONE_API_KEY)  # 환경 변수에서 가져오기

# Pinecone 클라이언트 인스턴스 생성
pc = Pinecone(
    api_key=MY_PINECONE_API_KEY
)

# 인덱스 이름 설정
index_name = 'movie-recommendation'

# 인덱스가 없으면 생성
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embedding_dimension,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',          # 사용 중인 클라우드 제공자 (예: 'aws' 또는 'gcp')
            region='us-east-1'    # 실제 환경 정보로 변경
        )
    )

# 인덱스에 연결
index = pc.Index(index_name)

Embedding Dimension: 4096


### 5. 메타데이터 준비

In [ ]:
# 임베딩 벡터의 ID 생성
ids = [str(i) for i in range(len(df))]

# 메타데이터 생성 시
metadata_list = []
for idx, row in df.iterrows():
    # 리뷰 필드의 길이를 700자로 제한
    review = row['Review']
    if review and isinstance(review, str):
        review = review[:400]  # 첫 400자만 사용
    else:
        review = ''

    # 모든 정보를 결합한 Content 필드 생성
    full_content = f"Title: {row['Title']}\n" \
                   f"Year: {row['Year']}\n" \
                   f"Genre: {row['Genre']}\n" \
                   f"Description: {row['Description']}\n" \
                   f"Director: {row['Director']}\n" \
                   f"Cast: {row['Cast']}\n" \
                   f"Review: {review}"

    metadata = {
        'Content': full_content,  # 모든 정보를 포함한 필드
        'Title': row['Title'],
        'Year': str(row['Year']),
        'Genre': row['Genre'],
        'Director': row['Director'],
        'Cast': row['Cast'],
        'Review': review  # 제한된 길이의 리뷰
    }
    metadata_list.append(metadata)

In [ ]:
metadata_list[0]

{'Content': 'Title: The Idea of You\nYear: 2023.0\nGenre: Comedy, Drama, Romance\nDescription: Solène, a 40-year-old single mom, begins an unexpected romance with 24-year-old Hayes Campbell, the lead singer of August Moon, the hottest boy band on the planet.\nDirector: Michael Showalter\nCast: Anne Hathaway, Nicholas Galitzine, Ella Rubin, Annie Mumolo\nReview: This film, as well as the reaction to it, is a wonderful example of modern hypocrisy:The story is about an adult woman who is attracted to a young guy, 16 years younger than her, young enough to be her son, whom she sees at his boys band concert, and from people reactions starts from the movie trailer and beyond, we get praise and stories that this is the destruction of the stigma about the happin',
 'Title': 'The Idea of You',
 'Year': '2023.0',
 'Genre': 'Comedy, Drama, Romance',
 'Director': 'Michael Showalter',
 'Cast': 'Anne Hathaway, Nicholas Galitzine, Ella Rubin, Annie Mumolo',
 'Review': 'This film, as well as the react

### 6. 임베딩 및 메타데이터 업로드

In [ ]:
# 업로드 배치 크기 설정
batch_size = 100

# 업로드할 데이터 준비
vectors = []
for i in range(len(ids)):
    vectors.append((ids[i], embeddings[i], metadata_list[i]))

# 배치로 업로드
for i in tqdm(range(0, len(vectors), batch_size)):
    batch = vectors[i:i+batch_size]
    index.upsert(vectors=batch)

100%|██████████| 94/94 [05:41<00:00,  3.64s/it]


### 7. 인덱스 상태 확인

In [ ]:
# 인덱스의 통계 정보 확인
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9342}},
 'total_vector_count': 9342}


### 8. LangChain과 Upstage 모델들 설정

In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatUpstage(api_key=MY_UPSTAGE_API_KEY, model="solar-pro")

messages = [
    HumanMessage(
        content="Can you recommend me a good science fiction movie?"
    )
]

response = llm.invoke(messages)
print(response)

content='Absolutely! I\'d recommend "Interstellar." It\'s a fantastic science fiction movie with a gripping story about space exploration.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 20, 'total_tokens': 54, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-pro-preview-240910', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-c949e131-391f-4936-bdfc-9ea08f34074b-0' usage_metadata={'input_tokens': 20, 'output_tokens': 34, 'total_tokens': 54}


In [ ]:
llm

ChatUpstage(client=<openai.resources.chat.completions.Completions object at 0x7fac94f01360>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fac94f02ad0>, model_name='solar-pro', model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar')

In [ ]:
from langchain_upstage import UpstageEmbeddings

# UpstageEmbeddings 인스턴스 생성
embed_model = UpstageEmbeddings(
    api_key=MY_UPSTAGE_API_KEY,
    model="solar-embedding-1-large-query"
)

In [ ]:
embed_model

UpstageEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7facdb661ba0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7fac8f7f86d0>, model='solar-embedding-1-large-query', dimensions=None, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar', embedding_ctx_length=4096, embed_batch_size=10, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, http_client=None, http_async_client=None)

### 9. Pinecone 벡터 스토어 설정

In [ ]:
# 필요한 라이브러리 임포트
from langchain.vectorstores import Pinecone as PineconeVectorStore

# 벡터 스토어 객체 생성
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embed_model,
    text_key='Content'  # 검색할 메타데이터 필드: 모든 정보를 담은 'Content' 필드 사용
)

<ipython-input-25-ec473bcf0122>:5: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = PineconeVectorStore(


In [ ]:
vectorstore

### 10. 사용자 정보 데이터 생성

In [ ]:
# 사용자 정보 딕셔너리 생성
user_info = {
    'born': 1999,
    'preferred_genres': ['Sci-Fi', 'Action'],
    'watched_movies': ['Inception', 'The Matrix'],
    'preferred_actors': ['Leonardo DiCaprio', 'Keanu Reeves']
}

# 사용자 정보를 문자열로 변환
user_info_str = f"Born: {user_info['born']}\n" \
                f"Preferred Genres: {', '.join(user_info['preferred_genres'])}\n" \
                f"Watched Movies: {', '.join(user_info['watched_movies'])}\n" \
                f"Preferred Actors: {', '.join(user_info['preferred_actors'])}"

In [ ]:
user_info_str

'Born: 1999\nPreferred Genres: Sci-Fi, Action\nWatched Movies: Inception, The Matrix\nPreferred Actors: Leonardo DiCaprio, Keanu Reeves'

### 11. 유사도 검색 및 결과 확인

- metadata_list 하나가 대략 약 190개의 단어 × 1.5 = 285 토큰 정도

In [82]:
# 사용자 쿼리 설정
query = "강력한 마법사가 등장하고 스펙타클한 모험이 펼쳐지는 판타지 영화 추천해줘"

In [83]:
# 유사도 검색 수행
results = vectorstore.similarity_search(query, k=10) # llm input으로 들어갈 수 있는 최대 토큰 수를 고려하여 k개 설정, solar-pro preview Context length: 4096

# 결과 출력
for result in results:
    print(f"Title: {result.metadata['Title']}")
    print(f"Year: {result.metadata['Year']}\n")
    print(f"Genre: {result.metadata['Genre']}\n")
    print(f"Director: {result.metadata['Director']}\n")
    print(f"Cast: {result.metadata['Cast']}\n")
    print(f"Description: {result.page_content}\n")

Title: Sorceress
Year: 2019.0

Genre: Action, Adventure, Fantasy

Director: Jack Hill

Cast: Leigh Harris, Lynette Harris, Roberto Nelson, David Millbern

Description: Title: Sorceress
Year: 2019.0
Genre: Action, Adventure, Fantasy
Description: Sword and sorcery story of the cosmic struggle between white magic and black magic.
Director: Jack Hill
Cast: Leigh Harris, Lynette Harris, Roberto Nelson, David Millbern
Review: I initially purchased this movie, along with two others, in a used bookstore's `Our Owner Is Dead Sale,' for the hefty price of one dollar. One must beg the question, of whether perhaps I got ripped off.Sorceress is bad. Astronomically bad. Not the kind of bad that we're used to. No, not mediocre Hollywood bad. It's so bad, it's almost a religious experience.The plot boils down to this: King/wizar

Title: Harry Potter and the Sorcerer's Stone
Year: 2002.0

Genre: Adventure, Family, Fantasy

Director: Chris Columbus

Cast: Daniel Radcliffe, Rupert Grint, Emma Watson, Ric

In [ ]:
results

[Document(metadata={'Cast': 'Daniel Radcliffe, Rupert Grint, Emma Watson, Richard Harris', 'Director': 'Chris Columbus', 'Genre': 'Adventure, Family, Fantasy', 'Review': 'I enjoyed this movie immensely. But, like "The Phantom Menace," I\'ve had a very hard time viewing it objectively. There was so much anticipation leading up to its release, I simply enjoyed the experience of being there. Having read all four books in the series a few times each, I am overly familiar with the events in the story. As I watched the movie, my continuing thought was "How well will the n', 'Title': "Harry Potter and the Sorcerer's Stone", 'Year': '2002.0'}, page_content='Title: Harry Potter and the Sorcerer\'s Stone\nYear: 2002.0\nGenre: Adventure, Family, Fantasy\nDescription: An orphaned boy enrolls in a school of wizardry, where he learns the truth about himself, his family and the terrible evil that haunts the magical world.\nDirector: Chris Columbus\nCast: Daniel Radcliffe, Rupert Grint, Emma Watson, R

### 12. 답변 생성을 위한 프롬프트 보강


(구)이전엔 모든 정보 다 쿼리로 넣었음 -> (현)제목, 장르, 리뷰, 설명만 넣음
- 리뷰 데이터 제한 없음, k=12: 11000 토큰
- 리뷰 데이터 700 제한, k=10: 5257 토큰
- 리뷰 데이터 600 제한, k=10: 4859 토큰
- 리뷰 데이터 500 제한, k=10: 4441 토큰

In [69]:
def augment_prompt(query: str, results, user_info_str):
    source_knowledge = "\n\n".join([
        f"Movie {idx+1}:\n"
        f"Title: {res.metadata['Title']}\n"
        f"Genre: {res.metadata['Genre']}\n"
        f"Review: {res.metadata['Review']}\n"
        f"Description: {res.page_content}"
        for idx, res in enumerate(results)
    ])

    augmented_prompt = f"""You are a movie recommendation system. Based on the user's query and the provided movie options, please recommend the top 5 movies that best match the user's preferences.

User Query:
{query}

User Info:
{user_info_str}

Here are some movie options:
{source_knowledge}

Please select the top 5 movies that best match the user's request, and provide their titles and a brief description in the following format:

Based on your request, I recommend the following movies:
1. Title - Brief Description
2. Title - Brief Description
3. Title - Brief Description
...

Please make sure the descriptions are concise and do not include any additional commentary."""
    return augmented_prompt

### 13. LLM을 활용하여 추천 결과 생성

In [80]:
# 보강된 프롬프트 생성
prompt = augment_prompt(query, results, user_info_str)

# LLM을 사용하여 응답 생성
response = llm(prompt)

# 응답 출력
print(response.content)

Based on your request, I recommend the following movies:

1. Harry Potter and the Sorcerer's Stone - A magical adventure about a young wizard named Harry Potter who discovers his true heritage and attends a school of witchcraft and wizardry.

2. The Sorcerer's Apprentice - An introverted physics prodigy must train in the art of sorcery to prevent the return of an evil sorceress.

3. Warcraft - A few human heroes and dissenting Orcs must attempt to stop the true evil behind a war between Orcs and humans on the planet Azeroth.

4. Willow - A young farmer is chosen to undertake a perilous journey to protect a special baby from an evil queen.

5. The Lord of the Rings: The Fellowship of the Ring - A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.
